In [1]:
import numpy as np

from astropy.cosmology import Planck18 as cosmo
from astropy import units as u

In [2]:
z          = 7.13              
L_CII      = 6.1e8                 * u.Lsun               
F_nu_0     = 0.06                  * u.mJy            
T_0        = 91.86                 * u.K    
kappa_0 = 10.41                    * u.cm**2 / u.g
y_Z        = 2                     * u.M_sun
nu_SN      = (1 / 53)              / u.M_sun
T_CMB      = cosmo.Tcmb0 * (1 + z)
g_z        = (1 + z) / cosmo.luminosity_distance(z).to(u.cm)**2

In [3]:
# Fiduciella värden
y         = (3.0 - 1.5) / 2 + 1.5
Sigma_SFR = 15     * u.M_sun / (u.kpc**2 * u.yr) 
M_star    = 1.0e10 * u.M_sun                     

# Ekv (15) i Sommovigo et al. (2021) med det antagna värdet av M_star
M_d_max   = (2.0  * u.M_sun) * (M_star / (53.0 * u.M_sun)) 

In [4]:
def CII(kappa_s, Z):

    # Ekv (1) i Sommovigo et al. (2021)
    D = (1/162) * Z 
    
    # Ekv (7) i Sommovigo et al. (2021)
    C = 32.47 * (u.M_sun / u.yr / u.kpc**2)**0.29
    alpha_CII = (C * kappa_s**(-5/7) * (y**2) * (Sigma_SFR**(-0.29))) * (u.M_sun / u.Lsun)

    # Ekv (14) i Sommovigo et al. (2021)
    A = (g_z * alpha_CII * D * L_CII * kappa_0).to(u.dimensionless_unscaled)
    F_tilde = 0.98e-16 * (F_nu_0 / u.mJy)
    
    # Ekv (13) i Sommovigo et al. (2021)
    B_T_CMB = 1.0 / (np.exp(T_0 / T_CMB) - 1.0)
    f = B_T_CMB + F_tilde / A
    
    # Ekv (12) i Sommovigo et al. (2021)
    T_d =  T_0 / np.log(1 + 1 / f)

    # Från det andra stycket i sektion 3.2 av Bakx et al. (2021)
    M_d = D * alpha_CII * L_CII

    # Villkor (i) i Bakx et al. (2021)
    if M_d > M_d_max:
        T_d = np.nan * u.K 
    
    # Från fotnot (3) i Bakx et al. (2021) och Dayal et al. (2010)
    SFR_FIR = 1.73 * 10**(-10) * M_d*(T_d / (6.73 * u.K))**6 * u.yr**(-1)

    # Ekv (8) i De Looze et al. (2014)
    SFR_CII = 10**(-7.06) * L_CII * (u.M_sun / (u.Lsun * u.yr))

    # Villkor (ii) i Bakx et al. (2021)
    if SFR_FIR > 10 * SFR_CII:
        T_d = np.nan * u.K

    return T_d.value, M_d.value

In [5]:
N_samples = 10000

k_s = np.random.uniform(1, 50, N_samples)
Z   = np.random.uniform(0.2, 1.0, N_samples)
T_d = np.zeros(N_samples)
m_d = np.zeros(N_samples)

# Det sista stycket i sektion 3.2 från Bakx et al. (2021) 
for i in range(N_samples):
    T_d[i], m_d[i] = CII(k_s[i], Z[i])

In [6]:
median_T_d = np.nanmedian(T_d)
lower_T_d  = np.nanpercentile(T_d, 16)
upper_T_d  = np.nanpercentile(T_d, 84)

median_m_d = np.nanmedian(m_d)
lower_m_d  = np.nanpercentile(m_d, 16)
upper_m_d  = np.nanpercentile(m_d, 84)

print("T_d  [Median]: {:.1f} K".format(median_T_d))
print("(-1σ) - (+1σ): {:.1f} - {:.1f} K\n".format(lower_T_d, upper_T_d))
print("m_d  [Median]: {:.1e} Msun".format(median_m_d))
print("(-1σ) - (+1σ): {:.1e} - {:.1e} Msun".format(lower_m_d, upper_m_d))

T_d  [Median]: 39.7 K
(-1σ) - (+1σ): 32.1 - 48.2 K

m_d  [Median]: 1.7e+07 Msun
(-1σ) - (+1σ): 9.1e+06 - 3.7e+07 Msun
